# Burgers' Equation

[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-pbs/PhiFlow/blob/develop/examples/grids/Burgers.ipynb)

Burgers' equation is a partial differential equation consisting of an advection term and a diffusion term acting on a vector field $v$ (velocity). It reads

$$\frac{\partial v}{\partial t} = \nu \frac{\partial^2 v}{\partial x^2} - v \frac{\partial v}{\partial x}.$$

In this example, we simulate Burgers' equation in 1D and 2D.

In [1]:
%pip install --quiet phiflow
from phi.jax.flow import *
# from phi.flow import *  # If JAX is not installed. You can use phi.torch or phi.tf as well.
from tqdm.notebook import trange

We define a time step as consisting of diffusion and advection, according to the PDE above.

In [26]:
@jit_compile
def step(v, dt=.5):
    v = diffuse.implicit(v, 0.1, dt=dt)
    v = advect.semi_lagrangian(v, v, dt=dt)
    return v

### 1D Simulation

After setting the initial and boundary conditions, we repeatedly call `step` using the `iterate` shorthand.

In [24]:
v0 = CenteredGrid(Noise(smoothness=1.5), PERIODIC, x=64, bounds=Box(x=64))
v_trj = iterate(step, batch(time=100), v0)
plot(v_trj, animate='time')

### 2D Simulation

By adding a `y` dimension to our `v0` grid, we run the simulation in 2D.

In [27]:
v0 = CenteredGrid(Noise(vector='x,y'), PERIODIC, x=64, y=64, bounds=Box(x=40, y=20))
v_trj = iterate(step, batch(time=100), v0)
plot(v_trj.as_points(), animate='time')